In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
#read the 2 df
dailyplay = pd.read_excel('data.xlsx', sheetname = 'dailyplay')
promotions = pd.read_excel('data.xlsx', sheetname = 'promotions')


In [4]:
#merge the 2 df
tot = pd.merge(dailyplay, promotions, how = 'left', left_on = ['Date'], right_on = ['Date'])

In [7]:
#I aggregate the data to day level so that every row in the df is a day
Rev = tot.groupby('Date')['Revenue'].sum()
Players = tot.groupby('Date')['Playerid'].count()

Rev = Rev.to_frame(name=None)
Players =Players.to_frame(name=None)
Date = tot['Date'].unique()
Promo = tot.groupby('Date')['Promo'].first().to_frame(name=None)

tot_new = pd.DataFrame(Date, columns=['Date'])
tot_new = pd.merge(tot_new, Players, how = 'left', left_on = ['Date'], right_index = True)
tot_new = pd.merge(tot_new, Rev, how = 'left', left_on = ['Date'], right_index = True)
tot_new = pd.merge(tot_new, Promo, how = 'left', left_on = ['Date'], right_index = True)

tot_new = (tot_new.sort_values(by=['Date'])
            .rename(columns={"Playerid": "Players"})
            )
#I also add day of the week to the df as I believe day of the week influences 
#promotion performance on revenue generated    
tot_new['Dayofweek'] = tot_new['Date'].dt.dayofweek
tot_new = tot_new.set_index('Date')

#this is the aggregated df
tot_new

,Players,Revenue,Promo,Dayofweek
Date,,,,
2016-10-01,707,5211,NaN,5
2016-10-02,1455,10386,A,6
2016-10-03,1520,12475,B,0
2016-10-04,1726,14414,NaN,1
2016-10-05,2134,20916,NaN,2
2016-10-06,1316,12996,B,3
2016-10-07,1287,11929,B,4
2016-10-08,1548,10072,NaN,5
2016-10-09,1448,12016,B,6


In [9]:
#I need all the columns to be numbers for the regression
conditions = [
    (tot_new['Promo'] == 'A'),
    (tot_new['Promo'] == 'B'),]
choices = ['1', '2']
tot_new['Promo_num'] = np.select(conditions, choices, default='0')

#X is the df with the independent variables
X = tot_new[['Players', 'Promo_num', 'Dayofweek']]

#y is the df with the dependent variable
y = tot_new['Revenue']

#This is where the model is built
X = sm.add_constant(X)
est = sm.OLS(y.astype(float), X.astype(float)).fit()

#The following will show a summary of the model where there are 4 coefficients 
#(the constant + 3 coefficients for the 3 independent variables)
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Revenue   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     373.4
Date:                Tue, 19 Dec 2017   Prob (F-statistic):           1.57e-76
Time:                        17:29:29   Log-Likelihood:                -1656.7
No. Observations:                 182   AIC:                             3321.
Df Residuals:                     178   BIC:                             3334.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4176.9827    740.739     -5.639      0.000   -5638.743   -2715.222
Players        9.5542      0.351     27.253      0.000       8.862      10.246
Promo_num   2081.8708    197.528     10.540      0.000    1692.072    2471.670
Dayofweek   -202.8544     87.326     -2.323      0.021    -375.181     -30.528
==============================================================================
Omnibus:                       79.847   Durbin-Watson:                   1.626
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              371.895
Skew:                           1.627   Prob(JB):                     1.75e-81
Kurtosis:                       9.201   Cond. No.                     7.95e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.95e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
#I need to create the df with the dates I want to predict the revenue using the model 
#I have just created
X_predict = [{'Date': '03/04/2017', 'Promo': '', 'Players': 3000},
         {'Date': '04/04/2017',  'Promo': 'A', 'Players': 4000},
         {'Date': '05/04/2017',  'Promo': '',  'Players': 4000},
        {'Date': '06/04/2017',  'Promo': 'B', 'Players': 5000},
        {'Date': '07/04/2017',  'Promo': 'A', 'Players': 6000},
        {'Date': '08/04/2017',  'Promo': 'B', 'Players': 6000},
        {'Date': '09/04/2017',  'Promo': '', 'Players': 7000}]
X_predict = pd.DataFrame(X_predict)
X_predict['Date'] =  pd.to_datetime(X_predict['Date'])
X_predict['Dayofweek'] = X_predict['Date'].dt.dayofweek

conditions = [
    (X_predict['Promo'] == 'A'),
    (X_predict['Promo'] == 'B'),]
choices = ['1', '2']
X_predict['Promo_num'] = np.select(conditions, choices, default='0')
X_predict = X_predict.set_index('Date')

#X_tab is the df with the data I need to predict the revenue 
X_tab = X_predict[['Players', 'Promo_num', 'Dayofweek']]
X_tab = sm.add_constant(X_tab)

#the below generate the predicted revenue using the model created above and the X_tab df
predictions = est.predict(exog=X_tab.astype(float))

pred = predictions.to_frame().rename(columns={ 0:"Predicted Revenue"})
X_predict = pd.merge(X_predict, pred, how = 'left', left_index = True, right_index = True)
X_predict


,Players,Promo,Dayofweek,Promo_num,Predicted Revenue
Date,,,,,
2017-03-04,3000,,5,0,23471.369817
2017-04-04,4000,A,1,1,35918.866252
2017-05-04,4000,,3,0,33431.286679
2017-06-04,5000,B,6,2,46540.673363
2017-07-04,6000,A,1,1,55027.282500
2017-08-04,6000,B,4,2,56500.590225
2017-09-04,7000,,0,0,62702.474159


In [16]:
# with the below the Confidence Intervals for the predicted values of revenue are calculated
inter = est.get_prediction(X_tab.astype(float))
inter = inter.summary_frame(alpha=0.05)[['mean_ci_lower','mean_ci_upper']]
inter.rename(columns={ 'mean_ci_lower':'CI lower', 'mean_ci_upper':'CI upper'})

,CI lower,CI upper
Date,,
2017-03-04,22241.318937,24701.420696
2017-04-04,34366.287754,37471.444750
2017-05-04,31706.820616,35155.752743
2017-06-04,43997.973183,49083.373542
2017-07-04,52123.515749,57931.049250
2017-08-04,53444.302257,59556.878194
2017-09-04,59090.696543,66314.251775
